# Secure Notebook Coding

## Prereq Libraries

### Install Libraries

Needed for Big Query and WML

In [ ]:
!pip install --upgrade google-cloud-bigquery[bqstorage,pandas] pyarrow ibm-watson-machine-learning

### Load Libraries

Dependencies for Examples below

In [ ]:
import json
import pandas as pd
import pyarrow

from google.cloud import bigquery
from google.oauth2 import service_account

## Insecure Examples

The following examples from our platform of insecure methods.

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [3]:
api_key = 'INSERT YOUR API KEY HERE'
location = 'us-south'

In [4]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Connection to Data Connections

Various data connections have degrees of insecurity in their **insert to code** functions in CPD and CPDaaS.

#### Google Big Query

Insert to code does only supports **insert credentials**.  As a result, users may bypass our code population and build from Google's examples potentially exposing credentials, by persisting the credentials JSON into Git repos.

#### IBM Cloud Object Store

IBM Cloud Object Store's insert to code function exposes API keys.  

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
def __iter__(self): return 0
# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_01da3b8d07aa40ca85ec5cee0637167f = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='KEYID-EXAMPLE',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')
body = client_01da3b8d07aa40ca85ec5cee0637167f.get_object(Bucket='MYBUCKET',Key='MYDATA.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df = pd.read_csv(body)
df.head()

## Secure Examples

Illustration of the above but implemented in a more secure fashion.

### Connection to WML 

Reference Key Options. 

 - `dotenv` doesn't seem to work on CPD.  Needs more testing.
 - `external file` Import credentials from a resident file.  Add to .gitignore.
 - `getpass` Prompt in the notebook execution for password.
 - `security vault` CPD Credentials Vault **Preferred**

#### External File
Parse an external file hidden from Github

In [66]:
import secrets
api_key = secrets.api_key

#### Getpass

Password Prompt in a Notebook

In [60]:
import getpass

api_key = getpass.getpass()

 ·····


#### Security Vault

Create new secret in the CPD Security Vault

In [2]:
# Define Secret (Note Insecure Example)
secret = {
    'secret_name':'example_ibm_api_key',
    'description':'IBM Cloud API Key',
    'secret':{'apikey':'INSERT YOUR KEY HERE'},
    'type':'credentials',
    'vault_name':'internal'}

##### Create Vault Secret

Store a secret in the vault so we can retrieve later for IBM Cloud API Key

In [3]:
# @hidden_cell

import os
import json
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

url = "https://ibm-nginx-svc/zen-data/v1/secrets"
token = os.environ.get('USER_ACCESS_TOKEN')

headers = {
    'Authorization': 'Bearer {}'.format(token),
    'Content-Type': 'application/json',
}

payload = json.dumps(secret)
response = requests.post(
    url,
    headers=headers,
    data=payload,
    verify=False,
)
print(response.text)

{"_messageCode_":"success","message":"Successfully added secret"}



##### Retrieve API Key

Retrieve the API Key from the created secret

In [4]:
url_secret_ref = "{}/{}::{}".format(url, secret['vault_name'], secret['secret_name'])

response = requests.get(
    url_secret_ref,
    headers=headers,
    verify=False,
)
print(response.text)

jsonResponse = response.json()
api_key = jsonResponse['data']['secret']['apikey']
print(api_key)

{"_messageCode_":"success","data":{"secret_name":"example_ibm_api_key","vault_name":"internal","type":"credentials","description":"IBM Cloud API Key","secret":{"apikey":"INSERT YOUR KEY HERE"},"created_at":"2023-09-11T21:36:54.653681Z","updated_at":"2023-09-11T21:36:54.653681Z"},"message":"Successfully retrieved secret"}

INSERT YOUR KEY HERE


In [ ]:
# Deprecated with CPD 4.x

#from icpd_core import icpd_util
#import json#

# You can request to view references to all secrets stored in the internal vault
#my_internal_vault_secret_references = icpd_util.get_my_secret_references()


### Connection to Data Connections

`@hidden_cell` can help to mask credentials, but has failings. Cells tagged as such should provide the following;

```# The code was removed by Watson Studio for sharing.```

#### Google Big Query

Leverage `project-lib` to hide credentials and base off of Watson Studio Connection Flow

In [ ]:

# @hidden_cell
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.

from project_lib import Project
project = Project.access()
BQ_IBM_MizzouTest_credentials = project.get_connection(name="BIG-QUERY-CONNECTION")


In [ ]:
json_account_info = json.loads(
    BQ_IBM_MizzouTest_credentials["credentials"]
)  # convert JSON to dictionary
credentials = service_account.Credentials.from_service_account_info(json_account_info)

client = bigquery.Client(credentials=credentials)

#### DB2 WoC

DB2 Is already secure

In [ ]:
# @hidden_cell

from project_lib import Project
project = Project.access()

IBM_DB2_WOC_metadata = project.get_connection(name="IBM DB2 WOC")

import os, jaydebeapi, pandas as pd

IBM_DB2_WOC_url = 'jdbc:db2://{}:{}/{}:sslConnection=true;'.format(
    IBM_DB2_WOC_metadata['host'],
    IBM_DB2_WOC_metadata.get('port', 50000),
    IBM_DB2_WOC_metadata['database']
)

IBM_DB2_WOC_connection = jaydebeapi.connect(
    'com.ibm.db2.jcc.DB2Driver',
    IBM_DB2_WOC_url,
    [IBM_DB2_WOC_metadata['username'],IBM_DB2_WOC_metadata['password']]
)
   
query = 'SELECT * FROM "EDW"."MYTABLE"'
data_df_1 = pd.read_sql_query(query, con=IBM_DB2_WOC_connection)
data_df_1.head()

# After use, close the database connection with the following code:
# IBM_DB2_WOC_connection.close()


## Appendix

In [51]:
import requests

headers = {
    'Authorization': 'Bearer {}'.format(token),
}

response = requests.get('https://ibm-nginx-svc/zen-data/v1/vaults/internal/secrets', headers=headers, verify=False)
print(response.text)

{"_messageCode_":"success","data":[{"secret_name":"oracle_db_login_credentials","vault_name":"internal","reference":"internal::oracle_db_login_credentials","type":"credentials"}],"message":"Successfully retrieved list of secrets"}



In [25]:
!curl -k -X POST \
  https://data-cpd-data.roks-sandbox-b2b6f340f1d04bfbf7bdc7a67f68010f-0000.us-south.containers.appdomain.cloud/zen-data/v1/secrets \
  -H "Authorization: Bearer $USER_ACCESS_TOKEN" \
  -H 'Content-Type: application/json' \
  -d '{"secret_name": "oracle_db_login_credentials", "description": "These are my login credentials for the Oracle instance with SSL", "secret": {"username": "testuser", "password": "PASSWORD"},"type": "credentials","vault_name": "internal"}'

{"_messageCode_":"success","message":"Successfully added secret"}


In [49]:
!curl -k -X GET https://data-cpd-data.roks-sandbox-b2b6f340f1d04bfbf7bdc7a67f68010f-0000.us-south.containers.appdomain.cloud/zen-data/v1/vaults/internal/secrets 
    -H "Authorization: Bearer $USER_ACCESS_TOKEN"

<html>
<head><title>401 Authorization Required</title></head>
<body>
<center><h1>401 Authorization Required</h1></center>
<hr><center>openresty</center>
</body>
</html>
